In [11]:
import torch

# Replace 'path_to_your_file.pt' with the actual path to your .pt file
file_path = '/proj/sourasb-220503/FedMEM/dataset/r3_mem_ResNet50FC_features/NEUR16_scan1_img01_day22_S.pt'

# Load the data
data = torch.load(file_path)
print(data.shape)
data

torch.Size([1024, 14, 14])


tensor([[[0.0436, 0.0524, 0.0730,  ..., 0.0716, 0.1524, 0.0000],
         [0.0597, 0.0426, 0.1641,  ..., 0.0309, 0.0000, 0.0000],
         [0.1100, 0.0910, 0.0389,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.1121, 0.1374, 0.2056,  ..., 0.0000, 0.1092, 0.4703],
         [0.1712, 0.1892, 0.2487,  ..., 0.1524, 0.2307, 0.2432],
         [0.0916, 0.1942, 0.1454,  ..., 0.8572, 0.4561, 0.4674]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1122, 0.4022],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0738, 0.1464],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0014, 0.1057,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0513,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.1898,  ..., 0.0421, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.

In [4]:
from PIL import Image
import torchvision.transforms as transforms


In [12]:
image_path = '/proj/sourasb-220503/FedMEM/dataset/r3_refined/NEUR16_scan1_img01_day22_S.jpg'

# Open the image
image = Image.open(image_path)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 256x256 pixels
    transforms.ToTensor(),          # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize pixel values (example for RGB)
                         std=[0.229, 0.224, 0.225])
])

# Apply the transformations to the image
tensor_image = transform(image)
print(tensor_image.shape)
tensor_image

torch.Size([3, 224, 224])


tensor([[[ 0.1768, -0.1486, -0.2856,  ...,  1.2214,  1.2214,  1.1872],
         [ 1.1358,  0.4508, -0.1828,  ...,  1.2043,  1.2043,  1.1529],
         [ 1.9920,  1.2043,  0.0227,  ...,  1.1700,  1.1700,  1.1358],
         ...,
         [-0.9877, -0.9534, -0.9363,  ..., -0.7650, -0.7822, -0.7650],
         [-0.9020, -0.9192, -0.9192,  ..., -0.8164, -0.7993, -0.7822],
         [-0.9192, -0.9534, -0.9363,  ..., -0.7308, -0.7479, -0.9020]],

        [[ 0.2402, -0.0924, -0.2325,  ...,  1.5007,  1.5007,  1.5357],
         [ 1.2206,  0.5203, -0.1275,  ...,  1.4832,  1.4832,  1.5182],
         [ 2.1134,  1.2906,  0.0826,  ...,  1.4482,  1.4482,  1.4657],
         ...,
         [-0.8627, -0.8978, -0.8803,  ..., -0.7752, -0.7752, -0.7752],
         [-0.7927, -0.8102, -0.8102,  ..., -0.7752, -0.7577, -0.7577],
         [-0.8452, -0.7927, -0.7752,  ..., -0.8102, -0.8277, -0.7402]],

        [[ 0.4439,  0.1128, -0.0267,  ...,  2.6226,  2.6226,  2.6226],
         [ 1.4200,  0.7228,  0.0779,  ...,  2

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset

In [19]:
data = torch.rand([1024, 14, 14])  # Simulated data; replace with your actual tensor
labels = torch.randint(0, 10, (1024,))  # Example target labels for a 10-class problem
data_upsampled = F.interpolate(data.unsqueeze(1), size=(224, 224), mode='bilinear', align_corners=False)



print(data_upsampled.shape)


data_upsampled = data_upsampled.repeat(1, 3, 1, 1)


print(data_upsampled.shape)



torch.Size([1024, 1, 224, 224])
torch.Size([1024, 3, 224, 224])


In [20]:
dataset = TensorDataset(data_upsampled, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Load a pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer to match the number of classes in your dataset (e.g., 10)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # Assuming 10 classes

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop (simplified)
for epoch in range(10):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

/home/x_souba/anaconda3/envs/personalized_fl/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/x_souba/anaconda3/envs/personalized_fl/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/x_souba/anaconda3/envs/personalized_fl/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available dev

Epoch 1, Loss: 2.217625856399536


KeyboardInterrupt: 